In [1]:
import numpy as np
from ax.api.client import Client
from ax.api.configs import RangeParameterConfig

In [2]:
client = Client()

In [3]:
# Define six float parameters x1, x2, x3, ... for the Hartmann6 function, which is typically evaluated on the unit hypercube
parameters = [
    RangeParameterConfig(
        name="x1", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x2", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x3", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x4", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x5", parameter_type="float", bounds=(0, 1)
    ),
    RangeParameterConfig(
        name="x6", parameter_type="float", bounds=(0, 1)
    ),
]

client.configure_experiment(parameters=parameters)

In [4]:
metric_name = "hartmann6" # this name is used during the optimization loop in Step 5
objective = f"-{metric_name}" # minimization is specified by the negative sign

client.configure_optimization(objective=objective)

In [5]:
# Hartmann6 function
def hartmann6(x1, x2, x3, x4, x5, x6):
    alpha = np.array([1.0, 1.2, 3.0, 3.2])
    A = np.array([
        [10, 3, 17, 3.5, 1.7, 8],
        [0.05, 10, 17, 0.1, 8, 14],
        [3, 3.5, 1.7, 10, 17, 8],
        [17, 8, 0.05, 10, 0.1, 14]
    ])
    P = 10**-4 * np.array([
        [1312, 1696, 5569, 124, 8283, 5886],
        [2329, 4135, 8307, 3736, 1004, 9991],
        [2348, 1451, 3522, 2883, 3047, 6650],
        [4047, 8828, 8732, 5743, 1091, 381]
    ])

    outer = 0.0
    for i in range(4):
        inner = 0.0
        for j, x in enumerate([x1, x2, x3, x4, x5, x6]):
            inner += A[i, j] * (x - P[i, j])**2
        outer += alpha[i] * np.exp(-inner)
    return -outer

hartmann6(0.1, 0.45, 0.8, 0.25, 0.552, 1.0)

np.float64(-0.4878737485613134)

In [6]:
for _ in range(10): # Run 10 rounds of trials
    # We will request three trials at a time in this example
    trials = client.get_next_trials(max_trials=3)

    for trial_index, parameters in trials.items():
        x1 = parameters["x1"]
        x2 = parameters["x2"]
        x3 = parameters["x3"]
        x4 = parameters["x4"]
        x5 = parameters["x5"]
        x6 = parameters["x6"]

        result = hartmann6(x1, x2, x3, x4, x5, x6)

        # Set raw_data as a dictionary with metric names as keys and results as values
        raw_data = {metric_name: result}

        # Complete the trial with the result
        client.complete_trial(trial_index=trial_index, raw_data=raw_data)

        print(f"Completed trial {trial_index} with {raw_data=}")

[WARNING 05-14 14:58:48] ax.api.client: 3 trials requested but only 2 could be generated.


Completed trial 0 with raw_data={'hartmann6': np.float64(-0.5053149917022333)}
Completed trial 1 with raw_data={'hartmann6': np.float64(-1.4059616089424354)}
Completed trial 2 with raw_data={'hartmann6': np.float64(-0.008868626772064313)}
Completed trial 3 with raw_data={'hartmann6': np.float64(-0.40354312756609867)}
Completed trial 4 with raw_data={'hartmann6': np.float64(-2.4607113182919647)}


[WARNING 05-14 14:58:50] ax.api.client: 3 trials requested but only 1 could be generated.


Completed trial 5 with raw_data={'hartmann6': np.float64(-2.6820591724123366)}
Completed trial 6 with raw_data={'hartmann6': np.float64(-2.5889257834528383)}
Completed trial 7 with raw_data={'hartmann6': np.float64(-0.022318256252344352)}
Completed trial 8 with raw_data={'hartmann6': np.float64(-0.007837026978328547)}
Completed trial 9 with raw_data={'hartmann6': np.float64(-2.2101280053832135)}
Completed trial 10 with raw_data={'hartmann6': np.float64(-0.32271680373105094)}
Completed trial 11 with raw_data={'hartmann6': np.float64(-1.588249731633073)}
Completed trial 12 with raw_data={'hartmann6': np.float64(-3.089262001307097)}
Completed trial 13 with raw_data={'hartmann6': np.float64(-0.344222228523341)}
Completed trial 14 with raw_data={'hartmann6': np.float64(-0.5585402722513332)}
Completed trial 15 with raw_data={'hartmann6': np.float64(-2.716396685047889)}
Completed trial 16 with raw_data={'hartmann6': np.float64(-0.2529091786270197)}
Completed trial 17 with raw_data={'hartmann6

In [7]:
client.summarize()

,trial_index,arm_name,trial_status,generation_node,hartmann6,x1,x2,x3,x4,x5,x6
0,0,0_0,COMPLETED,CenterOfSearchSpace,-0.505315,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
1,1,1_0,COMPLETED,Sobol,-1.405962,0.209322,0.348396,0.176615,0.355805,0.169381,0.498638
2,2,2_0,COMPLETED,Sobol,-0.008869,0.975172,0.539649,0.631677,0.990506,0.801456,0.923267
3,3,3_0,COMPLETED,Sobol,-0.403543,0.725384,0.085092,0.294095,0.162165,0.560789,0.697798
4,4,4_0,COMPLETED,Sobol,-2.460711,0.458988,0.776833,0.780441,0.558662,0.414049,0.130362
5,5,5_0,COMPLETED,MBM,-2.682059,0.317026,0.855098,0.788784,0.529770,0.237049,0.000000
6,6,6_0,COMPLETED,MBM,-2.588926,0.375224,1.000000,1.000000,0.502882,0.503191,0.000000
7,7,7_0,COMPLETED,MBM,-0.022318,0.576510,0.309238,0.947235,1.000000,0.000000,0.000000
8,8,8_0,COMPLETED,MBM,-0.007837,0.000000,1.000000,0.832114,0.000000,0.906114,0.000000
9,9,9_0,COMPLETED,MBM,-2.210128,0.517083,0.931011,0.327408,0.474114,0.306003,0.000000
